In [1]:
from ibapi.client import *
from ibapi.wrapper import *
from ibapi.contract import Contract, ComboLeg  # Import Contract for orders and ComboLeg for building combo legs
from ibapi.order import Order                # Import Order for creating order objects
from ibapi.tag_value import TagValue           # Import TagValue for additional order parameters
from ibapi.ticktype import TickTypeEnum

import threading
import time

import logging

# Configure logging to output INFO-level messages on the console
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s"
)

In [2]:
class TestApp(EClient, EWrapper):
    
    def __init__(self):
        """
        Initialize the TestApp as both a client and a wrapper for IB API.
        """
        EClient.__init__(self, self)
        # This variable will hold the next valid order ID provided by IB.
        self.OrderId = None

    
    def nextValidId(self, orderId: int):
        """
        Callback method called when the IB API provides a valid order ID.
        We simply store it in an instance variable.
        """
        self.OrderId = orderId
        print(f"Next valid order ID stored: {self.OrderId}")

    
    def nextId(self):
        """
        Increment and return the next available request ID.
        Ensures that every request (live or historical) has a unique identifier.
        """
        self.OrderId += 1
        return self.OrderId


    def contractDetails(self, reqId: int, details: ContractDetails):
        # Directly extract the contract ID from the object
        self.con_id = details.contract.conId
        print("Extracted ConId:", self.con_id)

    
    def tickPrice(self, reqId, tickType, price, attrib):
        # Convert the tick type to a readable string
        tickStr = TickTypeEnum.toStr(tickType)
        
        # Check if this tick corresponds to the LAST traded price
        if tickStr == "LAST" or tickStr == "CLOSE":
            self.lastPrice = price  # Save the last price
            print(f"Saved last price for reqId {reqId}: {self.lastPrice}")
    
        # If desired, print other ticks:
        print(f"reqId: {reqId}, tickType: {tickStr}, price: {price}, attrib: {attrib}")


    def calculate_combo_limit(self, combo_legs, fluctuation_pct):
        """
        Calculate the net combo price and the adjusted limit price using a percentage cushion.
        
        Parameters:
          combo_legs : dict
          fluctuation_pct : float
              The percentage cushion to apply, for example, 0.05 for 5%.
        
        Returns:
          net_price: float
              The calculated net price.
          adjusted_limit: float
              The limit price adjusted by adding fluctuation_pct * |net_price|.
        """
        net_price = 0.0
        for i in range(len(combo_legs['contr'])):
            if combo_legs['action'][i] == 'BUY':
                net_price += combo_legs['ratio'][i] * combo_legs['price'][i]
            elif combo_legs['action'][i] == 'SELL':
                net_price -= combo_legs['ratio'][i] * combo_legs['price'][i]
    
        # Adjusted limit: add fluctuation amount based on absolute net price.
        adjusted_limit = net_price + fluctuation_pct * abs(net_price)
        
        return net_price, adjusted_limit
    

    def createComboOrder(self, adjusted_limit_price: float, quantity: int, combo_action: str, combo_legs: dict):
        """
        Create and submit a combo order with up to six legs using a dictionary for leg details.
        
        Parameters:
          adjusted_limit_price: float
              The limit price for the overall combo order—already adjusted by your cushion.
          quantity: int
              The total number of combo contracts to trade.
          combo_action: str
              'BUY' or 'SELL' action over all the combo
          combo_legs: dict
              A dictionary containing combo legs information with the following format:
              {
                  'contr': ['DB', 'INTC'],     # List of instrument symbols.
                  'action': ['BUY', 'SELL'],   # List of actions for each leg.
                  'ratio': [1, 1],             # List of multipliers for each leg.
                  'id': [13435352, 270639],    # List of contract IDs for each instrument.
                  'price': [26.85, 19.92]      # List of prices for each leg.
              }
        """
        
        # ------------------------------
        # Define the combo contract.
        # ------------------------------
        contract = Contract()
        # Form a descriptive symbol by joining the instruments with a comma.
        contract.symbol = ",".join(combo_legs['contr'])
        contract.secType = "BAG"         # "BAG" tells IB we're dealing with a combo order.
        contract.exchange = "SMART"      # Use IB's SMART routing.
        contract.currency = "USD"        # Currency set to USD.
        
        # Initialize the list for combo legs.
        contract.comboLegs = []
        
        # ------------------------------
        # Loop over the provided legs and create a ComboLeg for each.
        # ------------------------------
        num_legs = len(combo_legs['contr'])
        for i in range(num_legs):
            comboLeg = ComboLeg()
            # Assign the required contract ID from the dictionary.
            comboLeg.conId = combo_legs['id'][i]
            # Set the ratio (multiplier) for this leg.
            comboLeg.ratio = combo_legs['ratio'][i]
            # Set the specified action (e.g., "BUY" or "SELL"); don't change its case.
            comboLeg.action = combo_legs['action'][i]
            # Use the default exchange "SMART" (can be adjusted if needed).
            comboLeg.exchange = "SMART"
            # Append the constructed leg to the contract's combo legs list.
            contract.comboLegs.append(comboLeg)
        
        # ------------------------------
        # Create the overall combo order.
        # ------------------------------
        order = Order()
        order.orderId = self.OrderId      # Use the current valid order ID stored earlier.
        order.action = combo_action       # Overall action for the combo
        order.orderType = "LMT"           # Use a limit order.
        order.lmtPrice = round(adjusted_limit_price, 2)  # Round to the nearest 0.01. Use the provided (adjusted) limit price.
        order.totalQuantity = quantity    # Total number of combo contracts to trade.
        order.tif = "GTC"                 # Time-In-Force: Good Till Canceled.
        
        # ------------------------------
        # Add smart combo routing parameters.
        # ------------------------------
        order.smartComboRoutingParams = []
        order.smartComboRoutingParams.append(TagValue('NonGuaranteed', '1'))
        # This tag acknowledges you're using a non-guaranteed combo order.
        
        # ------------------------------
        # Submit the combo order.
        # ------------------------------
        print(f"Placing Combo Order (ID {order.orderId}): {order.action} {quantity} {combo_action} combo ({contract.symbol}) @ {adjusted_limit_price}")
        self.placeOrder(order.orderId, contract, order)

    
    def error(self, reqId, errorTime, errorCode, errorString, advancedOrderReject):
        """
        This method handles error messages from TWS.
        When an error is received (for example, if the request fails), it prints the error details.
        """
        print(f"Error - reqId: {reqId},  errorTime: {errorTime}, errorCode: {errorCode}, errorString: {errorString}, OrderReject: {advancedOrderReject}")

        
    def openOrder(self, orderId: OrderId, contract: Contract, order: Order, orderState: OrderState):
        """
        Callback method that prints out details when an order is opened or updated.
        """
        print(f"openOrder: OrderID: {orderId}, Contract: {contract}, Order: {order}, State: {orderState}")

    def orderStatus(self, orderId: OrderId, status: str, filled: float, remaining: float,
                    avgFillPrice: float, permId: int, parentId: int, lastFillPrice: float,
                    clientId: int, whyHeld: str, mktCapPrice: float):
        """
        Callback method providing real-time updates on the status of orders.
        """
        print(f"orderStatus: OrderID: {orderId}, Status: {status}, Filled: {filled}, Remaining: {remaining}, "
              f"AvgFillPrice: {avgFillPrice}, ParentID: {parentId}")

    def execDetails(self, reqId: int, contract: Contract, execution: Execution):
        """
        Callback method that provides detailed information on trade executions.
        """
        print(f"execDetails: ReqID: {reqId}, Contract: {contract}, Execution: {execution}")


In [3]:
port = 7496  # Typical port for connecting to TWS (7496 for IB Gateway live trading)
clientId = 8

# Create an instance of the TestApp and connect to TWS.
app = TestApp()
app.connect("127.0.0.1", port, clientId)

# Start the API processing loop in a separate thread so that it does not block the main thread.
threading.Thread(target=app.run).start()
time.sleep(1)  # Pause briefly to ensure a reliable connection before making requests
    

2025-05-07 09:42:38,157 [INFO] sent startApi
2025-05-07 09:42:38,171 [INFO] REQUEST startApi {}
2025-05-07 09:42:38,172 [INFO] SENDING startApi b'\x00\x00\x00\t\x00\x00\x00G2\x008\x00\x00'
2025-05-07 09:42:38,173 [INFO] ANSWER connectAck {}
2025-05-07 09:42:38,174 [INFO] ANSWER openOrderEnd {}
2025-05-07 09:42:38,242 [INFO] ANSWER managedAccounts {'accountsList': 'U18112846'}


Next valid order ID stored: 43
Error - reqId: -1,  errorTime: 1746603759642, errorCode: 2104, errorString: Market data farm connection is OK:cashfarm, OrderReject: 
Error - reqId: -1,  errorTime: 1746603759642, errorCode: 2104, errorString: Market data farm connection is OK:usfarm.nj, OrderReject: 
Error - reqId: -1,  errorTime: 1746603759643, errorCode: 2104, errorString: Market data farm connection is OK:eufarm, OrderReject: 
Error - reqId: -1,  errorTime: 1746603759643, errorCode: 2104, errorString: Market data farm connection is OK:usopt, OrderReject: 
Error - reqId: -1,  errorTime: 1746603759643, errorCode: 2104, errorString: Market data farm connection is OK:usfarm, OrderReject: 
Error - reqId: -1,  errorTime: 1746603759643, errorCode: 2106, errorString: HMDS data farm connection is OK:euhmds, OrderReject: 
Error - reqId: -1,  errorTime: 1746603759643, errorCode: 2106, errorString: HMDS data farm connection is OK:fundfarm, OrderReject: 
Error - reqId: -1,  errorTime: 174660375964

In [4]:
combo_legs = {'contr':['DB','INTC'], 'action':['BUY','SELL'], 'ratio':[1,1], 'id':[0,0], 'price':[0,0]}

# combo_legs = {'contr':['DB','T'], 'action':['BUY','BUY'], 'ratio':[1,1], 'id':[0,0], 'price':[0,0]}

combo_legs

{'contr': ['DB', 'INTC'],
 'action': ['BUY', 'SELL'],
 'ratio': [1, 1],
 'id': [0, 0],
 'price': [0, 0]}

In [5]:
combo_legs['contr'][1]

'INTC'

In [6]:
for i, combo_leg_contr in enumerate(combo_legs['contr']):
    mycontract = Contract()
    mycontract.symbol = combo_leg_contr
    mycontract.secType = "STK"              # 'STK' means stock.
    mycontract.currency = "USD"             # Currency is US Dollars.
    mycontract.exchange = "SMART"           # Use SmartRouting for best execution.
    # mycontract.primaryExchange = "NASDAQ"   # Optional: Defines the primary exchange.

    app.reqContractDetails(app.OrderId, mycontract)
    time.sleep(1)
    combo_legs['id'][i] = app.con_id

    app.reqMktData(app.nextId(), 
                   mycontract,
                   "4", # last price (?)
                   True, # snapshot
                   False, # regulatorySnapshot
                   []) # no additional options are being set.

    time.sleep(1)
    combo_legs['price'][i] = app.lastPrice


2025-05-07 09:42:39,243 [INFO] REQUEST reqContractDetails {'reqId': 43, 'contract': 2153210798688: ConId: 0, Symbol: DB, SecType: STK, LastTradeDateOrContractMonth: , Strike: , Right: , Multiplier: , Exchange: SMART, PrimaryExchange: , Currency: USD, LocalSymbol: , TradingClass: , IncludeExpired: False, SecIdType: , SecId: , Description: , IssuerId: Combo:}
2025-05-07 09:42:39,247 [INFO] SENDING reqContractDetails b'\x00\x00\x00(\x00\x00\x00\t8\x0043\x000\x00DB\x00STK\x00\x00\x00\x00\x00SMART\x00\x00USD\x00\x00\x000\x00\x00\x00\x00'
2025-05-07 09:42:39,339 [INFO] ANSWER contractDetailsEnd {'reqId': 43}


Extracted ConId: 13435352


2025-05-07 09:42:40,256 [INFO] REQUEST reqMktData {'reqId': 44, 'contract': 2153210798688: ConId: 0, Symbol: DB, SecType: STK, LastTradeDateOrContractMonth: , Strike: , Right: , Multiplier: , Exchange: SMART, PrimaryExchange: , Currency: USD, LocalSymbol: , TradingClass: , IncludeExpired: False, SecIdType: , SecId: , Description: , IssuerId: Combo:, 'genericTickList': '4', 'snapshot': True, 'regulatorySnapshot': False, 'mktDataOptions': []}
2025-05-07 09:42:40,256 [INFO] SENDING reqMktData b'\x00\x00\x00-\x00\x00\x00\x0111\x0044\x000\x00DB\x00STK\x00\x00\x00\x00\x00SMART\x00\x00USD\x00\x00\x000\x004\x001\x000\x00\x00'
2025-05-07 09:42:40,319 [INFO] ANSWER tickReqParams {'tickerId': 44, 'minTick': 0.01, 'bboExchange': 'a60001', 'snapshotPermissions': 3}
2025-05-07 09:42:40,321 [INFO] ANSWER marketDataType {'reqId': 44, 'marketDataType': 1}
2025-05-07 09:42:40,323 [INFO] ANSWER tickSize {'reqId': 44, 'tickType': 0, 'size': Decimal('400')}
2025-05-07 09:42:40,324 [INFO] ANSWER tickSize {'

reqId: 44, tickType: BID, price: 26.98, attrib: CanAutoExecute: 1, PastLimit: 0, PreOpen: 0
reqId: 44, tickType: ASK, price: 27.02, attrib: CanAutoExecute: 1, PastLimit: 0, PreOpen: 0
Saved last price for reqId 44: 26.74
reqId: 44, tickType: CLOSE, price: 26.74, attrib: CanAutoExecute: 0, PastLimit: 0, PreOpen: 0


2025-05-07 09:42:41,270 [INFO] REQUEST reqContractDetails {'reqId': 44, 'contract': 2153210800128: ConId: 0, Symbol: INTC, SecType: STK, LastTradeDateOrContractMonth: , Strike: , Right: , Multiplier: , Exchange: SMART, PrimaryExchange: , Currency: USD, LocalSymbol: , TradingClass: , IncludeExpired: False, SecIdType: , SecId: , Description: , IssuerId: Combo:}
2025-05-07 09:42:41,271 [INFO] SENDING reqContractDetails b'\x00\x00\x00*\x00\x00\x00\t8\x0044\x000\x00INTC\x00STK\x00\x00\x00\x00\x00SMART\x00\x00USD\x00\x00\x000\x00\x00\x00\x00'
2025-05-07 09:42:41,337 [INFO] ANSWER contractDetailsEnd {'reqId': 44}


Extracted ConId: 270639


2025-05-07 09:42:42,272 [INFO] REQUEST reqMktData {'reqId': 45, 'contract': 2153210800128: ConId: 0, Symbol: INTC, SecType: STK, LastTradeDateOrContractMonth: , Strike: , Right: , Multiplier: , Exchange: SMART, PrimaryExchange: , Currency: USD, LocalSymbol: , TradingClass: , IncludeExpired: False, SecIdType: , SecId: , Description: , IssuerId: Combo:, 'genericTickList': '4', 'snapshot': True, 'regulatorySnapshot': False, 'mktDataOptions': []}
2025-05-07 09:42:42,272 [INFO] SENDING reqMktData b'\x00\x00\x00/\x00\x00\x00\x0111\x0045\x000\x00INTC\x00STK\x00\x00\x00\x00\x00SMART\x00\x00USD\x00\x00\x000\x004\x001\x000\x00\x00'
2025-05-07 09:42:42,588 [INFO] ANSWER marketDataType {'reqId': 45, 'marketDataType': 1}
2025-05-07 09:42:42,591 [INFO] ANSWER tickReqParams {'tickerId': 45, 'minTick': 0.01, 'bboExchange': '9c0001', 'snapshotPermissions': 3}
2025-05-07 09:42:42,593 [INFO] ANSWER tickString {'reqId': 45, 'tickType': 45, 'value': '1746602888'}
2025-05-07 09:42:42,596 [INFO] ANSWER tickS

Saved last price for reqId 45: 20.13
reqId: 45, tickType: LAST, price: 20.13, attrib: CanAutoExecute: 0, PastLimit: 0, PreOpen: 0
reqId: 45, tickType: BID, price: 20.08, attrib: CanAutoExecute: 1, PastLimit: 0, PreOpen: 0
reqId: 45, tickType: ASK, price: 20.13, attrib: CanAutoExecute: 1, PastLimit: 0, PreOpen: 0


In [7]:
combo_legs

{'contr': ['DB', 'INTC'],
 'action': ['BUY', 'SELL'],
 'ratio': [1, 1],
 'id': [13435352, 270639],
 'price': [26.74, 20.13]}

In [8]:
fluctuation_pct = 0.01

net_price, adjusted_limit = app.calculate_combo_limit(combo_legs, fluctuation_pct)

print("Calculated net price: {:.2f}".format(net_price))
print("Adjusted limit price: {:.2f}".format(adjusted_limit))


Calculated net price: 6.61
Adjusted limit price: 6.68


In [9]:
app.createComboOrder(
    adjusted_limit_price=adjusted_limit, 
    quantity=1, 
    combo_action='BUY', 
    combo_legs=combo_legs
)

2025-05-07 09:42:43,313 [INFO] REQUEST placeOrder {'orderId': 45, 'contract': 2153210793696: ConId: 0, Symbol: DB,INTC, SecType: BAG, LastTradeDateOrContractMonth: , Strike: , Right: , Multiplier: , Exchange: SMART, PrimaryExchange: , Currency: USD, LocalSymbol: , TradingClass: , IncludeExpired: False, SecIdType: , SecId: , Description: , IssuerId: Combo:;13435352,1,BUY,SMART,0,0,,-1;270639,1,SELL,SMART,0,0,,-1, 'order': 2153210793744: 45,0,0: LMT BUY 1.000000@6.68 GTC}
2025-05-07 09:42:43,320 [INFO] SENDING placeOrder b'\x00\x00\x01\xb8\x00\x00\x00\x0345\x000\x00DB,INTC\x00BAG\x00\x00\x00\x00\x00SMART\x00\x00USD\x00\x00\x00\x00\x00BUY\x001\x00LMT\x006.68\x00\x00GTC\x00\x00\x00\x000\x00\x001\x000\x000\x000\x000\x000\x000\x000\x002\x0013435352\x001\x00BUY\x00SMART\x000\x000\x00\x00-1\x00270639\x001\x00SELL\x00SMART\x000\x000\x00\x00-1\x000\x001\x00NonGuaranteed\x001\x00\x000\x00\x00\x00\x00\x00\x00\x000\x00\x00-1\x000\x00\x00\x000\x00\x00\x000\x000\x00\x000\x00\x00\x00\x00\x00\x000\x00\

Placing Combo Order (ID 45): BUY 1 BUY combo (DB,INTC) @ 6.676099999999999


2025-05-07 09:42:47,849 [INFO] ANSWER tickSize {'reqId': 45, 'tickType': 8, 'size': Decimal('1198')}


Saved last price for reqId 45: 19.94
reqId: 45, tickType: CLOSE, price: 19.94, attrib: CanAutoExecute: 0, PastLimit: 0, PreOpen: 0


2025-05-07 09:42:51,364 [INFO] ANSWER tickSnapshotEnd {'reqId': 44}
2025-05-07 09:42:53,375 [INFO] ANSWER tickSnapshotEnd {'reqId': 45}


Error - reqId: 45,  errorTime: 1746603774851, errorCode: 163, errorString: The following order "ID:45" price exceeds  the Percentage constraint of 3%. Restriction is specified in Precautionary Settings of Global Configuration/Presets., OrderReject: 
